In [162]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

options = Options()
options.add_experimental_option(
    'prefs', 
    {
#         'profile.managed_default_content_settings.javascript': 2,
        'profile.managed_default_content_settings.images': 2,
        'profile.managed_default_content_settings.mixed_script': 2,
        'profile.managed_default_content_settings.media_stream': 2,
        'profile.managed_default_content_settings.stylesheets':2
    }
)

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [190]:
df = pd.read_excel("URLs.xlsx")
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [191]:
for x in df['URL']:  
    driver.get(x + '?q=413&dates=09.03.2024-10.03.2024&guests=2')
    
    hover_element = driver.find_element(
        By.XPATH, "//*[@id=':0']/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[1]/div[2]/div[1]/div[3]/div")
    
    # Наводим на него
    ActionChains(driver).move_to_element(hover_element).perform()
    html = driver.page_source
    
    n = 0
    while not('zentexttip-text' in html) or n > 500:
        ActionChains(driver).move_to_element(hover_element).perform()
        html = driver.page_source
        n += 1
        
    
    soup = bs(html, 'html.parser')
    sert = soup.find("div", {'class': 'zentexttip-text'})

    print(sert.text)

Сертификат 78/АА-004/029-1/2022. По данным из Федерального перечня туристских объектов.
Сертификат 77/АА47/160-2022. По данным из Федерального перечня туристских объектов.
Сертификат 78/АА-034/733-2022. По данным из Федерального перечня туристских объектов.
Сертификат АА-022/307-2021. По данным из Федерального перечня туристских объектов.
Сертификат 78/АА-97/316-2022. По данным из Федерального перечня туристских объектов.
Сертификат 78/АА-034/594-2021. По данным из Федерального перечня туристских объектов.


In [79]:
def Sertificate(url):
    try:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.get(url)
       
        # Close the date search window (if it pops up)
        try:
            element = driver.find_element(By.CLASS_NAME, "Button_content__1ypi3")
            element.click()
        except NoSuchElementException:
            pass

       
       # Wait for the hover element to be clickable with an increased timeout
        try:
            hover_element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//*[@id=':0']/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[1]/div[2]/div[1]/div[3]/div")))
        except TimeoutException:
            return None

        # Наводим на него
        ActionChains(driver).move_to_element(hover_element).perform()
        # И теперь он подгрузился в html

        # выгружаем в html. Потом кидаешь это в суп и разбираешь
        html = driver.page_source
        soup = bs(html, 'html.parser')

        sert = soup.find("div", {'class': 'zentexttip-text'})
        return sert.text
   
    except NoSuchElementException as e:
        # Handle the exception (optional)
        print(f"Error: {e}. Skipping element and continuing...")
        return None

In [80]:
Sertificate(df['URL'][0])

'Сертификат 78/АА-004/029-1/2022. По данным из Федерального перечня туристских объектов.'

In [ ]:

#проверка функции
Sertificate(url)


df2 = pd.DataFrame()
# Define column names
headers = ['URL', 'Sertificate']
# Create an empty DataFrame with headers
df2 = pd.DataFrame(columns=headers)
df2


counter=0
#цикл для заполнения пустого датафрейма df2 значениями о наличии или отсутствии на странице сертификата
for i, url in enumerate(df['URL']):
   
    df2.loc[i, 'URL'] = df.loc[i, "URL"]
    df2.loc[i, 'Sertificate'] = Sertificate(df.loc[i, "URL"])
   
    counter += 1
    if counter >= 5:
        break

df2